In [117]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [118]:
GESTURE_SIZE = 10

In [119]:
user = str(1)
df = pd.read_csv('../raw-data/vol' + user + '.csv')

In [120]:
df = df.sort_values(['FINGER','Timestamp'])  # Sort values by finger first, then timestamp

In [121]:
df = pd.get_dummies(df,columns=['BTN_TOUCH'],drop_first=True) # drop first to prevent perfect predictor (overfitting)

# Feature Extraction

In [122]:
features = pd.DataFrame()

In [123]:
def extract_features(gesture):
    gesture.insert(len(gesture.columns), "X_Speed", 0)
    gesture.insert(len(gesture.columns), "X_Acceleration", 0)
    gesture.insert(len(gesture.columns), "Y_Speed", 0)
    gesture.insert(len(gesture.columns), "Y_Acceleration", 0)
    gesture.insert(len(gesture.columns), "Speed", 0)
    gesture.insert(len(gesture.columns), "Acceleration", 0)
    gesture.insert(len(gesture.columns), "Jerk", 0)
    gesture.insert(len(gesture.columns), "Ang_V", 0)
    gesture.insert(len(gesture.columns), "Path_Tangent", 0)
    gesture['X_Speed'] = (gesture.X - gesture.X.shift(1)) / (gesture.Timestamp - gesture.Timestamp.shift(1))
    gesture['Y_Speed'] = (gesture.Y - gesture.Y.shift(1)) / (gesture.Timestamp - gesture.Timestamp.shift(1))
    gesture['Speed'] = np.sqrt((gesture.X_Speed ** 2) + (gesture.Y_Speed ** 2))
    gesture['X_Acceleration'] = (gesture.X_Speed - gesture.X_Speed.shift(1)) / (
                gesture.Timestamp - gesture.Timestamp.shift(1))
    gesture['Y_Acceleration'] = (gesture.Y_Speed - gesture.Y_Speed.shift(1)) / (
                gesture.Timestamp - gesture.Timestamp.shift(1))
    gesture['Acceleration'] = (gesture.Speed - gesture.Speed.shift(1)) / (
                gesture.Timestamp - gesture.Timestamp.shift(1))
    gesture['Jerk'] = (gesture.Acceleration - gesture.Acceleration.shift(1)) / (
                gesture.Timestamp - gesture.Timestamp.shift(1))
    gesture['Path_Tangent'] = np.arctan2((gesture.Y - gesture.Y.shift(1)), (gesture.X - gesture.X.shift(1)))
    gesture['Ang_V'] = (gesture.Path_Tangent - gesture.Path_Tangent.shift(1)) / (
                gesture.Timestamp - gesture.Timestamp.shift(1))

    return gesture


In [124]:
current_gesture = pd.DataFrame(columns=df.columns)

In [125]:
current_gesture

,Timestamp,X,Y,TOUCH_MAJOR,TOUCH_MINOR,TRACKING_ID,PRESSURE,FINGER,BTN_TOUCH_HELD,BTN_TOUCH_UP


In [126]:
i = 0

In [127]:
df

,Timestamp,X,Y,TOUCH_MAJOR,TOUCH_MINOR,TRACKING_ID,PRESSURE,FINGER,BTN_TOUCH_HELD,BTN_TOUCH_UP
0,0.000000,350,1404,11,-420,2526,38,0,0,0
1,0.032676,348,1406,11,-420,2526,38,0,1,0
2,0.042109,347,1408,11,-420,2526,38,0,1,0
3,0.049881,344,1411,11,-420,2526,38,0,1,0
4,0.058581,340,1416,11,-420,2526,38,0,1,0
...,...,...,...,...,...,...,...,...,...,...
59546,898.462922,708,1831,-420,-420,3509,23,1,1,0
59549,898.579649,708,1830,-420,-420,3509,23,1,1,0
59553,898.621411,710,1830,-420,-420,3509,23,1,1,0
59557,898.663272,710,1826,-420,-420,3509,23,1,1,0


In [115]:
for row in df.iteritems():
    print(row)

('Timestamp', 0          0.000000
1          0.032676
2          0.042109
3          0.049881
4          0.058581
            ...    
59546    898.462922
59549    898.579649
59553    898.621411
59557    898.663272
59560    898.745893
Name: Timestamp, Length: 59565, dtype: float64)
('X', 0        350
1        348
2        347
3        344
4        340
        ... 
59546    708
59549    708
59553    710
59557    710
59560    710
Name: X, Length: 59565, dtype: int64)
('Y', 0        1404
1        1406
2        1408
3        1411
4        1416
         ... 
59546    1831
59549    1830
59553    1830
59557    1826
59560    1825
Name: Y, Length: 59565, dtype: int64)
('TOUCH_MAJOR', 0         11
1         11
2         11
3         11
4         11
        ... 
59546   -420
59549   -420
59553   -420
59557   -420
59560   -420
Name: TOUCH_MAJOR, Length: 59565, dtype: int64)
('TOUCH_MINOR', 0       -420
1       -420
2       -420
3       -420
4       -420
        ... 
59546   -420
59549   -420
59553 

In [129]:
for row in df.iteritems():
    if i >= GESTURE_SIZE:  # todo: include lifting finger as condition? and switching fingers
        features = extract_features(current_gesture) # append extracted features to features
        current_gesture = pd.DataFrame(columns=current_gesture.columns)  # clear data but keep columns
        i = 0
        continue
    current_gesture.loc[len(current_gesture.index)] = row  # append row to current_gesture
    i += 1

ValueError: cannot set a row with mismatched columns

In [103]:
features

""


In [104]:
current_gesture

,Timestamp,X,Y,TOUCH_MAJOR,TOUCH_MINOR,TRACKING_ID,PRESSURE,FINGER,BTN_TOUCH_HELD,BTN_TOUCH_UP
1,Timestamp,Timestamp,Timestamp,Timestamp,Timestamp,Timestamp,Timestamp,Timestamp,Timestamp,Timestamp
2,X,X,X,X,X,X,X,X,X,X
3,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
4,TOUCH_MAJOR,TOUCH_MAJOR,TOUCH_MAJOR,TOUCH_MAJOR,TOUCH_MAJOR,TOUCH_MAJOR,TOUCH_MAJOR,TOUCH_MAJOR,TOUCH_MAJOR,TOUCH_MAJOR
5,TOUCH_MINOR,TOUCH_MINOR,TOUCH_MINOR,TOUCH_MINOR,TOUCH_MINOR,TOUCH_MINOR,TOUCH_MINOR,TOUCH_MINOR,TOUCH_MINOR,TOUCH_MINOR
6,TRACKING_ID,TRACKING_ID,TRACKING_ID,TRACKING_ID,TRACKING_ID,TRACKING_ID,TRACKING_ID,TRACKING_ID,TRACKING_ID,TRACKING_ID
7,PRESSURE,PRESSURE,PRESSURE,PRESSURE,PRESSURE,PRESSURE,PRESSURE,PRESSURE,PRESSURE,PRESSURE
8,FINGER,FINGER,FINGER,FINGER,FINGER,FINGER,FINGER,FINGER,FINGER,FINGER
9,BTN_TOUCH_HELD,BTN_TOUCH_HELD,BTN_TOUCH_HELD,BTN_TOUCH_HELD,BTN_TOUCH_HELD,BTN_TOUCH_HELD,BTN_TOUCH_HELD,BTN_TOUCH_HELD,BTN_TOUCH_HELD,BTN_TOUCH_HELD
10,BTN_TOUCH_UP,BTN_TOUCH_UP,BTN_TOUCH_UP,BTN_TOUCH_UP,BTN_TOUCH_UP,BTN_TOUCH_UP,BTN_TOUCH_UP,BTN_TOUCH_UP,BTN_TOUCH_UP,BTN_TOUCH_UP


In [24]:
current_gesture

,Timestamp,X,Y,TOUCH_MAJOR,TOUCH_MINOR,TRACKING_ID,PRESSURE,FINGER,BTN_TOUCH_HELD,BTN_TOUCH_UP
0,Timestamp,Timestamp,Timestamp,Timestamp,Timestamp,Timestamp,Timestamp,Timestamp,Timestamp,Timestamp
1,X,X,X,X,X,X,X,X,X,X
2,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
3,TOUCH_MAJOR,TOUCH_MAJOR,TOUCH_MAJOR,TOUCH_MAJOR,TOUCH_MAJOR,TOUCH_MAJOR,TOUCH_MAJOR,TOUCH_MAJOR,TOUCH_MAJOR,TOUCH_MAJOR
4,TOUCH_MINOR,TOUCH_MINOR,TOUCH_MINOR,TOUCH_MINOR,TOUCH_MINOR,TOUCH_MINOR,TOUCH_MINOR,TOUCH_MINOR,TOUCH_MINOR,TOUCH_MINOR
5,TRACKING_ID,TRACKING_ID,TRACKING_ID,TRACKING_ID,TRACKING_ID,TRACKING_ID,TRACKING_ID,TRACKING_ID,TRACKING_ID,TRACKING_ID
6,PRESSURE,PRESSURE,PRESSURE,PRESSURE,PRESSURE,PRESSURE,PRESSURE,PRESSURE,PRESSURE,PRESSURE
7,FINGER,FINGER,FINGER,FINGER,FINGER,FINGER,FINGER,FINGER,FINGER,FINGER
8,BTN_TOUCH_HELD,BTN_TOUCH_HELD,BTN_TOUCH_HELD,BTN_TOUCH_HELD,BTN_TOUCH_HELD,BTN_TOUCH_HELD,BTN_TOUCH_HELD,BTN_TOUCH_HELD,BTN_TOUCH_HELD,BTN_TOUCH_HELD
9,BTN_TOUCH_UP,BTN_TOUCH_UP,BTN_TOUCH_UP,BTN_TOUCH_UP,BTN_TOUCH_UP,BTN_TOUCH_UP,BTN_TOUCH_UP,BTN_TOUCH_UP,BTN_TOUCH_UP,BTN_TOUCH_UP
